# Cipher

```C#
// I worked here: https://dotnetfiddle.net
// Original C# code to understand what's going on

// C# .NET 4.7.2
using System;
using System.IO;
using System.Security.Cryptography;
using System.Linq;
using System.Text;

public class Program {
    public static string text = "Catalyst";
    public static string passPhrase = "peekabeyoufoundme";

    public static void Main() {    
        string encr = Encrypt(text, passPhrase);
        string decr = Decrypt(encr, passPhrase);
        
        Console.WriteLine("Encrypted: " + encr);
        Console.WriteLine("Decrypted: " + decr);
    }

    private static byte[] Generate256BitsOfRandomEntropy() {
        // It's supposed to generate crypto-resistant entropy,
        // but that's not why we're here.
        var zeros = new byte[32];  // 32 Bytes will give us 256 bits of zeros.
        return zeros;
    }

    public static string Encrypt(string plainText, string passPhrase)
    {
        byte[] array = Generate256BitsOfRandomEntropy();   // May be any [32] bytes -> salt
        byte[] array2 = Generate256BitsOfRandomEntropy();  // May be any [32] bytes -> rgbIV
        byte[] bytes = Encoding.UTF8.GetBytes(plainText);
        byte[] bytes2 = new Rfc2898DeriveBytes(passPhrase, array, 1000).GetBytes(32);
        string result;

        // Console.WriteLine("array: " + string.Join(", ", array));

        using (RijndaelManaged rijndaelManaged = new RijndaelManaged()) {
            rijndaelManaged.BlockSize = 256;
            rijndaelManaged.Mode = CipherMode.CBC;
            rijndaelManaged.Padding = PaddingMode.PKCS7;
            using (ICryptoTransform cryptoTransform = rijndaelManaged.CreateEncryptor(bytes2, array2)) {
                using (MemoryStream memoryStream = new MemoryStream()) {
                    using (CryptoStream cryptoStream = new CryptoStream(memoryStream, cryptoTransform, CryptoStreamMode.Write)) {
                        cryptoStream.Write(bytes, 0, bytes.Length);
                        cryptoStream.FlushFinalBlock();
                        byte[] inArray = array.Concat(array2).ToArray<byte>().Concat(memoryStream.ToArray()).ToArray<byte>();
                        memoryStream.Close();
                        cryptoStream.Close();
                        result = Convert.ToBase64String(inArray);
                    }
                }
            }
        }
        return result;
    }

    public static string Decrypt(string cipherText, string passPhrase) {
        byte[] array = Convert.FromBase64String(cipherText);
        byte[] salt = array.Take(32).ToArray<byte>();
        byte[] rgbIV = array.Skip(32).Take(32).ToArray<byte>();
        byte[] array2 = array.Skip(64).Take(array.Length - 64).ToArray<byte>();
        byte[] bytes = new Rfc2898DeriveBytes(passPhrase, salt, 1000).GetBytes(32);
        string @string;

        // Console.WriteLine(string.Join(", ", array));

        using (RijndaelManaged rijndaelManaged = new RijndaelManaged()) {
            rijndaelManaged.BlockSize = 256;
            rijndaelManaged.Mode = CipherMode.CBC;
            rijndaelManaged.Padding = PaddingMode.PKCS7;
            using (ICryptoTransform cryptoTransform = rijndaelManaged.CreateDecryptor(bytes, rgbIV)) {
                using (MemoryStream memoryStream = new MemoryStream(array2)) {
                    using (CryptoStream cryptoStream = new CryptoStream(memoryStream, cryptoTransform, CryptoStreamMode.Read)) {
                        byte[] array3 = new byte[array2.Length];
                        int count = cryptoStream.Read(array3, 0, array3.Length);
                        memoryStream.Close();
                        cryptoStream.Close();
                        @string = Encoding.UTF8.GetString(array3, 0, count);
                    }
                }
            }
        }
        return @string;
    }
}
```

In [3]:
cipher = "1jbkcWr/gM7pq8omY3+Q17uRMawx3r0/93TjKMKL1f9foX4RdwigYj6x8zQWP+D+zs4/C0lgG9EL7kg14GEAFSvoRtvSQWo9n1t7d0t+dXGYlw1CLd68kvKm5C41tTOJp/M0gpxV/Y0C1RFN/YJZ9UPGBnl6TSHtiuHvT9hT9dumN8QKw99KroV4wu0EDt6wOZGsY7WgWPHOLi5Fpz9JeC6Axizsl3OYJ++CzJoQt8xv+1Ie47hBblo1796Sm5OGRrXxcpxbLLwd2UH0Hy0sCXA2Cbj9bqf0xiF5MHW2VfTL690KBCZU8jw5Vnwsr8jTd5esHMDsjiA3Oqe6GJAYyXkOBnTz1qjSy7GdGRkfe9u82EikSPZdgSK7fm2MBiu1c8t9LD9V/wHgwQ5GvOZUSnFP1o2KLN1ZM0ZzIcWsacaou0922KX9xIb0e+DtLuOHBuAoHa9Nrnn05CVGo5vQJC2EEBQahc6L7jKHT97jyK5jojayiQUCdsP/BBQWXHLSZ9OYAtRtIiHaLw6ZSK/8GZYmlRQWjRVEyzurMRcR081BNsmIUi4Ek2ituW9skORpp0nO/O0+/jFBy+F2o/VWQxY39WozUQV/7c8QYzRU/OM1q6JIdNfH/2JOgRBILOLFwtftq/9S2eDeec6d57WEPeVVh/6wAnm0+T6xh6B2wwa58G/+67dcH5fISaCC/KDmWIGYWUp5Bx5ZWjqROj+/tZd42pUGVT6xNmcPpbh54JdvEjqqdnha2eb+dvWsEj0eAMX45tzM43PN7Ch8+tV0mjLBZDQJs+bol7yNzXNJBEzfftAKraAkluxQDdIKcIkar9yCr4tSwzv4wug4M13X/gCrMqvVhiq0sPz6D3djHFKUvABSxaNcFNwG779kA5UMEGYY60XkzD5ZHi9QjA/uV1FcopIWOw1ZTVJKQGMqmcChRZK63DUX8IIB6+/BPuDhrMJtB0e7tZ0vZrjBAQtPLhyDrlbd/bLbkCE4oDaenDEaFyHMbzM3hFXy+tA6CInQldqNr0wYCB41h3W3GXL2epp97YmoOE8pyFBA7tk3jsmECvndEFqAYzksYAA/7O3uZnbxLbApAnq19d7A1M1kPHWhszJSwLAtQCraJTeiOO+5X3od/0rQ6Xk6p11plIiPR07R+5bDEcmvzsJKflcsN6IKh/WamM4HwMXB/fe4L5YsBVZakSWzgDs/fUwEvjeUk0kWtziddnMfoNun9uHZuSCwEghvEMzq7R4+HkzGTfvxl64ocA1cD7sOrSvJxqAnkyKmFh7NXV0UFZcRc8KPplNPVVBcrUODZj88qQMpBzj+9QMPeyDKqv3YXjHrGDN3Ha+T9nAX3+3imK85mqw36l1SWU9R8bR4pc/u+kk73Fck+poNPapwXvvwJ+fa1dKrkWvcpiik68hzhMLRK3WHraoiEoK/kvtq4SbceyrE7VxlCx+xKhX+RZfNgt2Oy0BxA6YT98bUKWjkE2MN1Vk01OBEAVNi/VCSgDCLbz3kG4oOYeylFKSBMqI+CNwYeiaE0fU4xYpvx/e54Uoehl+xDYH0jjbQga8ehwpENned/5xX7VJmvlOqGbVPS+AlugcIBU5EijpzLq+RjW7CWE8KuzPLHhbaF6zXPf5gjE/gbq2agD2S91kDQ7bJ8BVYy7SNWWRUHJAYEqFx56ow3s0PqA3W/wCyx3hfz4K5u1dhOIy9+xcQTS24pZL3LOJ+VoCZkAz4qoZOFbV9gE2hvgco0XLY2Gy5R8cyTMAscoLfOPR6wJ3uGt2YbhMF5nxzoQ4foKh48mJDcv9rahdclLS/4J4vVvm40goh8I+PWMP2kssB8Z1rQKyZKzuS2hnJtSH91Dg/Z/P13DQF6I4wK6CIycjTZEwU7Js0krXISBkfrGaJewCV+selvqXQy6JPhnZiY3MkD3tlBIbr2fb38TWgfSVOSsTMNNof2L2Cr+1nH4Zk0nM6ovKltIVCi5zS+dkZY2XgbtHXeIOZX7xBTBWG/NEnG3BS/FACo5tVKfRs1wkTYUYNZFAm2YHQQ4tla6//Z0Yi1PYaW7I2bi/1cpngIVNGVuc+D3m4znwYoyaRxU2OPBFiB5bAQ1UsT3ZJiM6gYdP2VdgmV1XNDBEJiNmlFTNIL4IuybXdK7zZMIa3N5ig6YzOpY6boHDHjjK5mm6rUo5WdiM5MiNpYZoLiobU8vMslz0srJ4R9rOIObmKH7EoKZzG10jnC8RKhopRKww31pW/xpriXPwk9SNOLG0upd9ds8ELTaD5B1Az1nl9CZH/NsgfdcdR/C7p13WNnYN58QViaZCoqqmaJjnbC0r7lSl3I8T+PpZ2tuzHDyx04RXt+WWyXHVUKW2FqFf+5iaFx63QFPSh4LPqTZZB2a0wPCasxpase0pLz/pLfatdZ1rkcTYPuvEdVOV9IBeRc13X/wzpe78Q1ze/3V8BuN9LTK1WfTOc781kOP37hvYaoZ5JjxpJOBdo+wKp8dJMW3Q++fcF7YZQClouVR7nZYkKL4/KlTQ5hsT5Zj5eJaiMb9lvbke7BFdaB//5Xty0lq2huP3pTNPbe/QakuRQYs+TTuGrpTVM/L7Mh4AoFFqXqeSVUw7aDtbOyOm29YL4XKuEF4JaMVX2gOmWaxxdeVOojtRTSgyHUbeAfBlVBDI+A/uMohW8ZVBQsNVZ1EUBI1pWlb831+d3ZIcPdO+/SQUEHSNVOkKY0TsPSO9CBOTyoD7+5ua4s6ZWPSYDmoeS7sxvHu8pwTofVzh2m2awm0fJ3DchfqSm39X1gBDQm3ekpcdbYn3CdoarNJnns+pT+qKv1EbQLdbhllItrg0ZieuyWusPXqbLLhO+C6/cv0kdjUzczrjpkmrEhtzRJmtiJg0nGffjZdTnFtxZSXFUjYkYeTgrASuFhnhVj2orDoTjauzgDfp7m5Tj2N/NgHjHzQrFe/6XFD2pxJkrkGVuzDSaoRBZ6t65CArkOLANd86HvKuia9mvwomcNpzc2tyUSgzUDIIaJ9gzBYkUmbtpFDXxZWoVwAlOqkUXCcRhdTw6BHsEYTIN6QhFPoJ63YluIn/S5qeaLa65f3UxTSwqgGctGeWAfOpNSQY3ChjGISfZJA4ocm2jDuVaNvs9kIs+FyAZByeIiWhWtgfBggtuR6ja0gMk4s7TACeZDT6v2Q7S2AiMsFsUF1rtbmqoxyM40jc419IuayBBsN2Hw1E2/KPDD7jlaHcapg3+1qrXUpDmXKPjYmPz0R8Fz8HlhiQPye4ckofn5dU+tC8NHKOjDjn4zKoMpDr90CD4ZDk8pTerP7F0xGH9/4v0t+eeTGAFaoWYQtNBIbPgol6bqH3HA4qJlvL6Vmal7w8rZPdmImlDqwXpvmi0Nd+rKaE4ZaxdtcaFSnGkLNgvMgt6/2NII2DFC7r/SVlUo8HidAx7DK+gm8b6TSVNFkEPhU3I2bSUFQYa308h3I7YmGTzQTDl2SCBUobuf+rTUAZ5Y9T80jQDhNpNbJFPxIyQ4A/f8TM9TzLirn+2CYWrgo6UvJ3CUSurWg=="
password = "peekabeyoufoundme"
text = "Hello world!"

In [4]:
# !pip3 install pprp==0.2.5

import pprp
import base64

def CBC_decrypt(key, s, iv, block_size=32):
    r = pprp.crypto_3.rijndael(key, block_size=block_size)

    i = 0
    for block in s:
        decrypted = r.decrypt(block)
        decrypted = xor(decrypted, iv)  
        iv = block

        yield decrypted
        i += 1

def CBC_encrypt(key, s, iv, block_size=32):
    r = pprp.crypto_3.rijndael(key, block_size=block_size)

    i = 0
    for block in s:
        len_ = len(block)
        if len_ < block_size:
            padding_size = block_size - len_
            block += (chr(padding_size) * padding_size).encode('ASCII')
        
        xored = xor(block, iv)
        encrypted = r.encrypt(xored)
        iv = encrypted

        yield encrypted
        i += 1

def xor(block, iv):
    result = [(a ^ b) for (a, b) in zip(block, iv)]
    return bytes(result)

def printbytes(name, array):
    print(name, ", ".join([str(i) for i in array]))

# Encryption
joke = "MADE/BY/CATALYST/42/"
joke = bytes(base64.b64decode(joke))

salt = joke + bytes([0]*(32 - len(joke)))
rgbIV = bytes([0]*32)
array2 = list(bytes("TEXT".encode("utf-8")))
kbytes = pprp.pbkdf2(password.encode('utf-8'), bytes(salt), 32, iterations=1000)

blocksize = 32
sg = pprp.data_source_gen(array2, blocksize)
dg = CBC_encrypt(kbytes, sg, rgbIV, blocksize)
encrypted = pprp.encrypt_sink(dg)

encrypted = base64.b64encode(salt + rgbIV + encrypted)
print(encrypted)
cipher = encrypted

# Decryption
array = list(bytearray(base64.b64decode(cipher)))
salt = array[:32]
rgbIV = array[32:64]
array2 = array[64:]
kbytes = pprp.pbkdf2(password.encode('utf-8'), bytes(salt), 32, iterations=1000)

blocksize = 32
sg = pprp.data_source_gen(array2, blocksize)
dg = CBC_decrypt(kbytes, sg, rgbIV, blocksize);
decrypted = pprp.decrypt_sink(dg, blocksize).decode("utf-8")

decrypted

b'MADE/BY/CATALYST/42/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABW9UFnavuNrJnPO/DEtQSUeqWF8dg9OTmwNOX76czqY'


'TEXT'

# Tests

In [8]:
cipher = "ATaPD7+HZQW5cvPMlLUw/RIqcBsxQC5p+9A9BgTcUJGJn/zZ1kiEbjFHgz3aDNl95RckBQ+0F/5iCnlweJdWR1ObCgzck6Df/zJpFnq44hNrKQXJuesYE25XfRYqbL3lO6BXnUEQy2uJtZCLTYltEPJYT2nVwweGF5xli4oK/6dp4E42VuiaGU2rTODEBLq8cPNezkTmQo1Yp4to2/8XS8Aut1e0XhnVntgVIvFlHKDxbpjJJDrbz95k8BGf0RWB3xyWMlIDJhjv+OcYowbrX2vq4wQsQuYqf8LPWxTEOXpla/g0bEXfqC5gUJI/MdGB1NAy6m5pvUhLCYVYfPqyREitTLurJmgyhNUGZUcWKPlte/m+sazHirfUsUQGf/sBPb0DJ6T238AbvPIwbVLaCozue+KB/a3Svn2lJPumVUtJX+PQXPj2Ttjc1CKstvzGiMkanin1Q4Hn1hKUlVcJaPGphSmiHwS2fNQksSe9NxjpXAnUImapoVBfbjIK1dW2ccqpVer8d45gv/53tH8nK4ju1Tl6oznQqfDxds5r/b8eRTVOxiqhtFBfpQVsL9+0ApocPBLvL+xwT2IbAAzsBny9uyPG3tyJ9CjuvhJgGijquUkoYYeXsIaL9IksZYf3elpwbUOj+0KkqBta5UzkbYlE/OjuIjEmBJrgdlFgupwmFgplaqEd7RXLtZtUuSawN59dT588oNorWSXQF434sKh2GpxHADahUFrqD69PQn2RCYwcT3fxkSyuS723I4ZDZPrfEioVZt6CO/ePXjdVBacqDtMv3qJHqcD5UaNrC9nckq7yxOl+O5BEXpqeahHfrsEY4MxqV9fftTpvnJlbatPYRmu8Ha/lYJglajDMYQIqrAJKvjAevqeTqaxg02xt1kJ36575V5TapwGdIzQEg/Lri1O5BLKQ7swmbrnDMRWa0+NvUWrJZKBZ5j5IdwWTrK6+FUm3Dj1GfVlW7vGjqRMsghD8QsdFWTHy77XohqVNreaqwGzu6QQe+aTn1KoaDY9hEoBUpBPNVRQsSbgXaoGdpV8wMQ7blCNgELyX+EvMyEUsPDu7erYvSQZgKpiA6skkda4e4CmuA7krJ1FdEVKykzCqK+zq7+1cg1T4rMre4vtRM5F+wbW2qcSrK++umXMzWxpDzNHrpMNCyoghrkOgLfO2XJokOstIeGc727I8BrtTfEal56l1Qedu2C508nIjHT2AFBZWzYwqZ39rY/42Mgep7dxIwPu7fktzVqykmATT9f+QAv3YnGRlx9wLCrOGPo98h8n/OXyv3GoSqk019R/3Lfg5K/P94zPMbf0WMMFialgmlNVrdF+O+cidZfkqC+rXJGH3/b3SA2K1cTk48B36Kh10lAkrUEgWl0p3pe73iA9P92fjgeVDD6ZUs3ASCK8O7Sm0WZEs6X9Dw5g+FZHwlrfziDrmixKsGPGSGs6zuMaTftQgUVYq6rbfwS4mrLk+t4uJ+cGhYFzHwbUJTTR11ps/P3VexdK4nmqRcjIiHrTVdgmV4LkrfvatKas1zaSbcywio9Ibz9kZEVt0bibLXEwDRYbBVM6xfxDhsZKwx1vJBSXQQ/hriOjVxs9H5dXoQLC4zB6cl2G9rWmZgX1xB2hOsPsmOyMhYzSXDy3u607MnG7b9t7wSw8TEsq2IV+MRZFk9i4AtHy3v2O67+wjoZbLIzgDE7D3N//q/D4Eg9TIjufN+O1IxlnIX4wtyGCem+slnu2hoqvtKMHm7AccnpR5OUAGUAypBC+enPSP2mJXRWbutdlvltBNqhsGqdLuqjk4DfUkyKb5xRDN1IfC7kwjXrQ7HFz9UXgxAlZOOlafaYlcRnefKF7cZud2ycfQRZNLV5RXGJYxl2YT2nSFDLdEap3CLV1kdE+z907uz2bZeT77qcxqJvTPQJvf1AmE41oPMmpxUioLhiB9oZQSCJywHxwiVcLk6cOlhtuWYjJ9H3NMSB2qzBJgaSv81TeDnG6fnvvrk0FpWpY0GtMA9u0KOqdNgWaHfB29b0aEPr98FuLzY/97ScKsO98/NkYky018rWC6XLxt+yxfrN3Ipu4VbnDIPFPjd7SCftbxQK7hTGW63+opkQ2h4dYhyjr0BaKSDxQXPuEAvMDFdUSX5dUxUsbyUyUZYqs0o13aMmanhpqeVbKln9vADAMF/Kg6qW6f0jYdIgNWI8jxHSsxm3kF3BwmO3Tio75ueu+RYkLDgcVMA3r1VzZGROzJywEAf+94rfA3DsIt0clyiS28k0nEtNgZkKGNiAR+BqTHlloAwHx6kPXKHx1iBLCnie2YwKI4ET5Mts5GQmFSNZBfzY3wRiukD8l8eqF8yA4b4/3lSOe8FJepu33bmNIJ1pxULlimFbfWyrxtVUWy/qU88A9yAHbntBVPAvXKpfpYb7yyZAyAff1Ogescl7lAc/fhuTRAokNiLqE2gLeM+Cd/OPks3j5lkKlLdvVL4EORnaEjpT5vw02ug6b45pC3ugew4MsBbnudp9zLnyPtZ1OzJmCVLz95OqYr5sgUTV1E1Na8sWA7Sb2wSE2X1F/jtWpdtMjXoF30wmqaE1aoCDCoDL/L0GqKDMpH0XSGUYXhc65lNziJqFXuKxvLvt23vnekhAejqcAVidPt0WABojFMju7L7SAusf8eaCSiwdWhhCMp0z9MlWjVNtljmzqrHYD2/wgs0GR2x5pqobm6ArouAcY9bOvh6JfCDxnIH+jdM3rO18FBZCWIEao82x2EdzPhYRkGMPixUvM28wQ4ZDJnm7OjgK6L1RQRyFmNB1rXTlk6CSW2aIspssAGL9lSmN2CDB1TG8+kpyf8jrzUssukKKR0/CduZ3JFYyYupKiLDM0+c6E1ynJeqAQevDeVV+DTyoGTTrLbB0n4qadq/Agij3zmWW18pEpy0zz/NLGjhFEMPXl/XZDTDJmdhzKyJ1mFZDddzRntJbthL05j44iIFhnFHhBNP1dn+oWergu8L8gbEKej6YTswwgRwi4fTDP6yDyxDxM8HYgZJGuBDEgZfM+iahf5K3Xrdpg/AzQKSztw8PRabegGFY6Wys961eS/tvImRGKWswaYrJAeT5O7rAiks4B9takbM8Hk3qNizZ21w0xJz2XXVVsuHRTbRnn+9tsm4w6E/CS2JsVrSpftZDrTj5/9WK+O/oaHE/1qtqSyrBl4QCTJEVFCyirXlYrWCTGeq/2xGvik0bQz1LgOT6//AqHsMoG11GxoBxh2x8ayPM7Hh8P1//GcbWD1lFmIgnJwKBFSLpVcz4InMyz+HyvZPwHEGDcUEKx+pWvwzA23dUgxMzJmwxC6vAmi8HnC4UZjb2QYSvRjFSll9bexlKQnTVL0cPQoiwX2zerzKviHoerLQ19gbVe43j0S342B0PLgoH3IxYNItO6/wBJcWfCq+MNgMDn9lzmNiVYWPwO8+3Tsozt31Ie4gpCWxbMYkb44t5PTbZPYUd/nm+Bb/+ErpV7KsCy+eYlmsgFgzFyQvIOwLVHx6BeTCdhvNjfpigDFnkgPvEJYCCYVVYpD6hnyvRVoePtQ7CxXtDoZX9V8WFf0+VFdOtwA56IyA5LyLIaPXDQrgBxzmBR3KnQJtw0SWOKOAiDIZQAvXvEtlqmr0b4Z8UEdE1zbCxEEVy50SzJwijB7UOSG5w=="

array = list(bytearray(base64.b64decode(cipher)))
salt = array[:32]
rgbIV = array[32:64]
array2 = array[64:]
kbytes = pprp.pbkdf2(password.encode('utf-8'), bytes(salt), 32, iterations=1000)

blocksize = 32
sg = pprp.data_source_gen(array2, blocksize)
dg = CBC_decrypt(kbytes, sg, rgbIV, blocksize);
decrypted = pprp.decrypt_sink(dg, blocksize).decode("utf-8")

decrypted[:64] + "..."


'{version:3.60.2,rng:297931346,hero_settings:{lastSaveTime:05/07/...'

In [6]:
decrypted = """{version:3.60.2,rng:297931346,hero_settings:{lastSaveTime:05/07/2024 13:36:00,playerName:simple one,bigHead:False},progress_flags:{flags:[show_workstation,show_items,show_stone,show_wood,rocky_plateau_1,show_banner,deadwood_valley_1,PATCH_ELEMENT_DAMAGE_v2100,PATCH_HAMMERS_v2110,PATCH_LONG_SWORDS_v2140]},quest_data:{available:[deadwood_valley,rocky_plateau],star_levels:{},aspiring_star_ids:[],aspiring_stars:[],has_seen:[find_shelter,rocky_plateau,build_door,deadwood_valley,build_workstation,craft_shovel,craft_hatchet,prospect_cliff],has_played:[rocky_plateau,find_shelter,build_door,deadwood_valley,build_workstation,craft_hatchet,craft_shovel,prospect_cliff],has_completed:[rocky_plateau,find_shelter,build_door,deadwood_valley,build_workstation,craft_hatchet,craft_shovel,prospect_cliff],stats:[],skullnata:05/05/2024 10:58:08},inventory_data:{Stone:3000000,Wood:30042000,itms:[{id:dirty_sword,da:{hI:True,rng:0}},{id:stones,da:{hI:True,rng:0}},{id:hatchet_lv1,da:{hI:True,rng:0}},{id:shovel_lv1,da:{hI:True,rng:0}},{id:sight_stone,da:{hI:True,rng:0}}],rightH:stones_lv1,itmsSeen:[sight_stone_lv1,stones_lv1,hatchet_lv1_lv1,shovel_lv1_lv1,dirty_sword_lv1]},cosmetics:{},treasure_factory:{uniqueDate:04/30/2024 12:10:33,crystalDate:04/30/2024 12:10:33,goldDate:04/20/2024 12:10:33},ui_state:{screen:QuestScreen,quest_scroll_y:0,workstation_scroll_y:0,items_scroll_y:1},shop_states:{ids:[],hasSeenShopkeeper:False,totalPurchases:0},crypt_intro:{timesPlayed:0,nextTreasureAvailableDate:05/05/2024 12:10:33},xp:{currentLevel:120,currentXP:0},ouroboros:{hasBeenTapped:False},utility_belt:{loadouts:[]},craft_book:{discovered_crafts:[]},achievements:{FOES_DEFEATED:1},mutator:{seed:1830886142},events:{sIds:[]},custom_quests:{ftueStep:UnlockBasicQuests,revealed:[],active:[],counter:0,completedCount:0,nextSpawnDate:05/05/2024 12:10:33,epicSpawnPending:False,basicQuestDate:05/05/2024 00:00:00},weekly_quest:{questCount:0,expiration:05/05/2024 12:10:33},goals:{skeleton_arm:{goal:-1,progress:0},blade_of_god:{goal:-1,progress:0},cult_mask:{goal:-1,progress:0},fire_talisman:{goal:-1,progress:0},aether_talisman:{goal:-1,progress:0}},subs:{},prom:{},leaderboards:{pId:""},mind_stone:{enabled:False,program:[// В рабочем состоянии Камень разума постоянно,// выполняет данные ему указания.,// Указания должны быть написаны на,// Камнескрипте.,// Узнайте больше на:,// StoneStoryRPG.com/stonescript,// - Мудрец Безерра,"",?hp < 7,"  activate potion",?loc = caves,"  equipL sword","  equipR shield","  ?foe = boss","    equip crossbow"],saveId:1}}"""

joke = "MADE/BY/CATALYST/42/"
joke = bytes(base64.b64decode(joke))

salt = joke + bytes([0]*(32 - len(joke)))
rgbIV = bytes([0]*32)
array2 = list(bytes(decrypted.encode("utf-8")))
kbytes = pprp.pbkdf2(password.encode('utf-8'), bytes(salt), 32, iterations=1000)

blocksize = 32
sg = pprp.data_source_gen(array2, blocksize)
dg = CBC_encrypt(kbytes, sg, rgbIV, blocksize)
encrypted = pprp.encrypt_sink(dg)

encrypted = base64.b64encode(salt + rgbIV + encrypted)
cipher = encrypted

encrypted[:64] + b"..."

b'MADE/BY/CATALYST/42/AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...'